In [2]:
import requests

api_key = "tEdVoKjbCzRuX1SnQPZyZYVzUlsZSrauLdxmMIWt"
url = f"https://api.nasa.gov/planetary/apod?api_key={api_key}"

response = requests.get(url)
data = response.json()

print(data)

{'date': '2024-02-24', 'explanation': "Intuitive Machines' robotic lander Odysseus has accomplished the first U.S. landing on the Moon since the Apollo 17 mission in 1972. Launched on a SpaceX rocket on February 15, the phone booth sized lander reached lunar orbit on the 21st and touched down on the lunar surface at 6:23 pm ET on February 22nd. Its landing region is about 300 kilometers north of the Moon's south pole, near a crater designated Malapert A. Resting on its side, the lander is presently collecting solar power and transmitting data back to the Intuitive Machines' mission control center in Houston. The mission marks the first commercial uncrewed landing on the Moon. Prior to landing, Odysseus’ camera captured this extreme wide angle image (landing legs visible at right) as it flew over Schomberger crater some 200 kilometers from its landing site. Odysseus was still about 10 kilometers above the lunar surface.", 'hdurl': 'https://apod.nasa.gov/apod/image/2402/im-moon-imageFeb2

In [3]:
url = f"https://api.nasa.gov/EPIC/api/natural?api_key={api_key}"

response = requests.get(url)
images = response.json()

# Print the URL of the latest image
if images:
    image_date = images[0]['date'].split()[0].replace('-', '/')
    image_name = images[0]['image']
    print(f"https://epic.gsfc.nasa.gov/archive/natural/{image_date}/png/{image_name}.png")

https://epic.gsfc.nasa.gov/archive/natural/2024/02/23/png/epic_1b_20240223002712.png


In [4]:
import ee
import requests
from PIL import Image

ee.Authenticate()

ee.Initialize()

In [56]:

point = ee.Geometry.Point([-74.011227, 40.706556])
rectangle = ee.Geometry.Rectangle([-125, 24, -66, 50])
startDate = ee.Date('2020-01-01')
endDate = ee.Date('2023-01-01')

# Load Landsat 8 imagery
landsat = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR') \
            .filterBounds(rectangle) \
            .filterDate(startDate, endDate) \
            .sort('CLOUD_COVER') \
            .first()

# Print the map URL
url = landsat.getThumbURL({'min': 0, 'max': 3000, 'dimensions': '1600x1600', 'bands': ['B4', 'B3', 'B2']})

# Download the image
response = requests.get(url)

# Save the image
save_path = 'image.jpg'
with open(save_path, 'wb') as file:
    file.write(response.content)

# Print the image
Image.open(save_path).show()


In [60]:
landsat.getInfo()

{'type': 'Image',
 'bands': [{'id': 'B1',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': -32768,
    'max': 32767},
   'dimensions': [7951, 8051],
   'crs': 'EPSG:32620',
   'crs_transform': [30, 0, 207585, 0, -30, 5220015]},
  {'id': 'B2',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': -32768,
    'max': 32767},
   'dimensions': [7951, 8051],
   'crs': 'EPSG:32620',
   'crs_transform': [30, 0, 207585, 0, -30, 5220015]},
  {'id': 'B3',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': -32768,
    'max': 32767},
   'dimensions': [7951, 8051],
   'crs': 'EPSG:32620',
   'crs_transform': [30, 0, 207585, 0, -30, 5220015]},
  {'id': 'B4',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': -32768,
    'max': 32767},
   'dimensions': [7951, 8051],
   'crs': 'EPSG:32620',
   'crs_transform': [30, 0, 207585, 0, -30, 5220015]},
  {'id': 'B5',
   'data_type': {'type': 'PixelType',
    'precision': 'in

In [48]:

class SattelieImage:
    def __init__(self, image_location:list, start_date:str, end_date:str):
        """image_location is a list of lat, longs. If you have just one then it will take a square image around that point.
          If you give two then it will make a rectange with those two points being the corners of the rectangle."""
        self.image_location = image_location
        self.start_date = start_date
        self.end_date = end_date
        self.url = None
        self.captured_image_date = None

        assert ee is not None, "Please authenticate and initialize Earth Engine"
        assert len(image_location) == 2 or len(image_location) == 4, "Please enter either two or four lat, longs"
        assert requests is not None, "Please install the requests library"
        assert Image is not None, "Please install the PIL library"
        
    
    def update_location(self, new_location):
        self.image_location = new_location
        self.url = None
    
    def update_dates(self, new_start_date, new_end_date):
        self.start_date = new_start_date
        self.end_date = new_end_date
        self.url = None
        
    def get_map_url(self):

        if len(self.image_location) == 2:
            rectangle = ee.Geometry.Point(self.image_location)
        elif len(self.image_location) == 4:
            rectangle = ee.Geometry.Rectangle(self.image_location)
        else:
            print("Please enter either two or four lat, longs")
            return None
        
        landsat = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR') \
            .filterBounds(rectangle) \
            .filterDate(self.start_date, self.end_date) \
            .sort('CLOUD_COVER') \
            .first()

        self.captured_image_date = landsat.date().format('YYYY-MM-dd').getInfo()

        url = landsat.getThumbURL({'min': 0, 'max': 3000, 'dimensions': '1600x1600', 'bands': ['B4', 'B3', 'B2']})
        self.url = url
        return url
    
    def download_image(self, url = None):
        if url is None:
            if self.url is None:
                url = self.get_map_url()
            else:
                url = self.url
        
        response = requests.get(url)
        save_path = 'image.jpg'
        with open(save_path, 'wb') as file:
            file.write(response.content)
        return save_path
    
    def show_image(self, path = None, describe_image_date = False):
        if path is None:
            path = self.download_image()
        Image.open(path).show()

        if describe_image_date:
            print(f"The image was captured on {self.captured_image_date}")
        return None
    
satImage = SattelieImage([-74.011227, 40.706556], '2021-01-01', '2023-01-01')
satImage.show_image(describe_image_date=True)

The image was captured on 2021-02-08


In [53]:
satImage.update_location([49.686641, 25.910484])
satImage.update_dates('2022-01-01', '2024-01-01')
satImage.show_image(describe_image_date=True)

EEException: Image.date: Parameter 'image' is required.